- [sequence-to-sequence learning in Keras](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html)


- [paper](https://proceedings.neurips.cc/paper_files/paper/2014/file/a14ac55a4f27472c5d894ec1c3c743d2-Paper.pdf)

In [1]:
!wget  https://www.manythings.org/anki/hin-eng.zip
!unzip -q hin-eng.zip /content/hin-eng.zip

--2023-12-17 12:44:46--  https://www.manythings.org/anki/hin-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 135012 (132K) [application/zip]
Saving to: ‘hin-eng.zip’

hin-eng.zip         100%[===================>] 131.85K  --.-KB/s    in 0.1s    

2023-12-17 12:44:46 (1.24 MB/s) - ‘hin-eng.zip’ saved [135012/135012]

caution: filename not matched:  /content/hin-eng.zip


In [2]:
#importing libraries
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [3]:
batch_s = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dimension = 256  # Latent dimensionality of the encoding space.
number_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = '/content/hin.txt'

In [8]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(number_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)


In [9]:
print('Number of input samples:', len(input_texts))
print('Number of target samples:', len(target_texts))

Number of input samples: 2979
Number of target samples: 2979


In [10]:
input_texts

['Wow!',
 'Duck!',
 'Duck!',
 'Help!',
 'Jump.',
 'Jump.',
 'Jump.',
 'Hello!',
 'Hello!',
 'Cheers!',
 'Cheers!',
 'Exhale.',
 'Exhale.',
 'Got it?',
 "I'm OK.",
 'Inhale.',
 'Inhale.',
 'Thanks!',
 'We won.',
 'Awesome!',
 'Come in.',
 'Get out!',
 'Go away!',
 'Goodbye!',
 'Perfect!',
 'Perfect!',
 'We lost.',
 'Welcome.',
 'Welcome.',
 'Have fun.',
 'Have fun.',
 'Have fun.',
 'I forgot.',
 'I forgot.',
 "I'll pay.",
 "I'm fine.",
 "I'm full.",
 "Let's go!",
 'Answer me.',
 'Birds fly.',
 'Excuse me.',
 'Fantastic!',
 'I fainted.',
 'I fear so.',
 'I laughed.',
 "I'm alone.",
 "I'm alone.",
 "I'm bored.",
 "I'm broke.",
 "I'm tired.",
 "It's cold.",
 'Well done!',
 'Who knows?',
 'Who knows?',
 'Who knows?',
 'Who knows?',
 'Wonderful!',
 'Birds sing.',
 'Come on in.',
 'Definitely!',
 "Don't move.",
 'Fire burns.',
 'Follow him.',
 'I can swim.',
 'I can swim.',
 'I love you.',
 'I love you.',
 'I love you.',
 'I love you.',
 'I love you.',
 "I'm coming.",
 "I'm hungry!",
 "I'm hu

In [11]:
target_texts

['\tवाह!\n',
 '\tझुको!\n',
 '\tबतख़!\n',
 '\tबचाओ!\n',
 '\tउछलो.\n',
 '\tकूदो.\n',
 '\tछलांग.\n',
 '\tनमस्ते।\n',
 '\tनमस्कार।\n',
 '\tवाह-वाह!\n',
 '\tचियर्स!\n',
 '\tसांस छोड़।\n',
 '\tसांस छोड़ो।\n',
 '\tसमझे कि नहीं?\n',
 '\tमैं ठीक हूँ।\n',
 '\tसांस ले।\n',
 '\tसांस लो।\n',
 '\tधन्यवाद!\n',
 '\tहम जीते।\n',
 '\tबहुत बढ़िया!\n',
 '\tअंदर आ जाओ।\n',
 '\tबाहर निकल जाओ!\n',
 '\tचले जाओ!\n',
 '\tख़ुदा हाफ़िज़।\n',
 '\tउत्तम!\n',
 '\tसही!\n',
 '\tहम हार गए।\n',
 '\tआपका स्वागत है।\n',
 '\tस्वागतम्।\n',
 '\tमज़े करना।\n',
 '\tमौज करना।\n',
 '\tमज़े करो।\n',
 '\tमैं भूल गया।\n',
 '\tमैं भूल गई।\n',
 '\tमैं पैसे दूंगा।\n',
 '\tमैं ठीक हूँ।\n',
 '\tमेरा पेट भर गया है।\n',
 '\tचलो चलें!\n',
 '\tमुझे जवाब दो।\n',
 '\tपंछी उड़ते हैं।\n',
 '\tमाफ़ कीजिए।\n',
 '\tबहुत ख़ूब!\n',
 '\tमैं बेहोश हो गया।\n',
 '\tखेद की बात है, लेकिन वैसा ही है।\n',
 '\tमैं हँसा।\n',
 '\tमैं अकेला हूँ ।\n',
 '\tमैं अकेली हूँ ।\n',
 '\tमैं बोर हो रहा हूँ।\n',
 '\tमेरा दीवालिया हो चुका है।\n',
 '\tमैं थक गया हूँ।\n',
 '

In [12]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [14]:
max_encoder_seq_length

107

In [13]:
max_decoder_seq_length

123

In [15]:
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of unique input tokens: 70
Number of unique output tokens: 90
Max sequence length for inputs: 107
Max sequence length for outputs: 123


In [16]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [17]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 "'": 4,
 ',': 5,
 '-': 6,
 '.': 7,
 '0': 8,
 '1': 9,
 '2': 10,
 '3': 11,
 '4': 12,
 '5': 13,
 '6': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'R': 36,
 'S': 37,
 'T': 38,
 'U': 39,
 'V': 40,
 'W': 41,
 'Y': 42,
 'a': 43,
 'b': 44,
 'c': 45,
 'd': 46,
 'e': 47,
 'f': 48,
 'g': 49,
 'h': 50,
 'i': 51,
 'j': 52,
 'k': 53,
 'l': 54,
 'm': 55,
 'n': 56,
 'o': 57,
 'p': 58,
 'q': 59,
 'r': 60,
 's': 61,
 't': 62,
 'u': 63,
 'v': 64,
 'w': 65,
 'x': 66,
 'y': 67,
 'z': 68,
 '€': 69}

In [18]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '"': 4,
 '(': 5,
 ')': 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 ':': 12,
 '?': 13,
 'A': 14,
 'B': 15,
 'I': 16,
 '|': 17,
 'ँ': 18,
 'ं': 19,
 'ः': 20,
 'अ': 21,
 'आ': 22,
 'इ': 23,
 'ई': 24,
 'उ': 25,
 'ऊ': 26,
 'ऋ': 27,
 'ए': 28,
 'ऐ': 29,
 'ऑ': 30,
 'ओ': 31,
 'औ': 32,
 'क': 33,
 'ख': 34,
 'ग': 35,
 'घ': 36,
 'च': 37,
 'छ': 38,
 'ज': 39,
 'झ': 40,
 'ञ': 41,
 'ट': 42,
 'ठ': 43,
 'ड': 44,
 'ढ': 45,
 'ण': 46,
 'त': 47,
 'थ': 48,
 'द': 49,
 'ध': 50,
 'न': 51,
 'प': 52,
 'फ': 53,
 'ब': 54,
 'भ': 55,
 'म': 56,
 'य': 57,
 'र': 58,
 'ल': 59,
 'व': 60,
 'श': 61,
 'ष': 62,
 'स': 63,
 'ह': 64,
 '़': 65,
 'ा': 66,
 'ि': 67,
 'ी': 68,
 'ु': 69,
 'ू': 70,
 'ृ': 71,
 'ॅ': 72,
 'े': 73,
 'ै': 74,
 'ॉ': 75,
 'ो': 76,
 'ौ': 77,
 '्': 78,
 '।': 79,
 '०': 80,
 '१': 81,
 '४': 82,
 '५': 83,
 '६': 84,
 '७': 85,
 '८': 86,
 '९': 87,
 '\u200d': 88,
 '€': 89}

### For one Hot encoding

In [19]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [20]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [21]:
decoder_target_data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0.

In [22]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dimension, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [23]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dimension, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [24]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [25]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_s,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
38/38 [==============================] - 6s 52ms/step - loss: 1.2953 - accuracy: 0.7846 - val_loss: 1.5441 - val_accuracy: 0.6848
Epoch 2/100
38/38 [==============================] - 1s 25ms/step - loss: 1.0159 - accuracy: 0.8072 - val_loss: 1.5355 - val_accuracy: 0.6785
Epoch 3/100
38/38 [==============================] - 1s 31ms/step - loss: 0.8783 - accuracy: 0.8085 - val_loss: 1.6064 - val_accuracy: 0.6873
Epoch 4/100
38/38 [==============================] - 1s 25ms/step - loss: 0.8454 - accuracy: 0.8091 - val_loss: 1.3676 - val_accuracy: 0.6873
Epoch 5/100
38/38 [==============================] - 1s 26ms/step - loss: 0.8213 - accuracy: 0.8091 - val_loss: 1.4574 - val_accuracy: 0.6873
Epoch 6/100
38/38 [==============================] - 1s 25ms/step - loss: 0.7959 - accuracy: 0.8090 - val_loss: 1.3672 - val_accuracy: 0.6649
Epoch 7/100
38/38 [==============================] - 1s 25ms/step - loss: 0.8554 - accuracy: 0.8028 - val_loss: 1.3101 - val_accuracy: 0.6873
Epoch 

In [26]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dimension,))
decoder_state_input_c = Input(shape=(latent_dimension,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [27]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [28]:
reverse_input_char_index

{0: ' ',
 1: '!',
 2: '"',
 3: '$',
 4: "'",
 5: ',',
 6: '-',
 7: '.',
 8: '0',
 9: '1',
 10: '2',
 11: '3',
 12: '4',
 13: '5',
 14: '6',
 15: '7',
 16: '8',
 17: '9',
 18: ':',
 19: '?',
 20: 'A',
 21: 'B',
 22: 'C',
 23: 'D',
 24: 'E',
 25: 'F',
 26: 'G',
 27: 'H',
 28: 'I',
 29: 'J',
 30: 'K',
 31: 'L',
 32: 'M',
 33: 'N',
 34: 'O',
 35: 'P',
 36: 'R',
 37: 'S',
 38: 'T',
 39: 'U',
 40: 'V',
 41: 'W',
 42: 'Y',
 43: 'a',
 44: 'b',
 45: 'c',
 46: 'd',
 47: 'e',
 48: 'f',
 49: 'g',
 50: 'h',
 51: 'i',
 52: 'j',
 53: 'k',
 54: 'l',
 55: 'm',
 56: 'n',
 57: 'o',
 58: 'p',
 59: 'q',
 60: 'r',
 61: 's',
 62: 't',
 63: 'u',
 64: 'v',
 65: 'w',
 66: 'x',
 67: 'y',
 68: 'z',
 69: '€'}

In [29]:
reverse_target_char_index

{0: '\t',
 1: '\n',
 2: ' ',
 3: '!',
 4: '"',
 5: '(',
 6: ')',
 7: ',',
 8: '-',
 9: '.',
 10: '0',
 11: '1',
 12: ':',
 13: '?',
 14: 'A',
 15: 'B',
 16: 'I',
 17: '|',
 18: 'ँ',
 19: 'ं',
 20: 'ः',
 21: 'अ',
 22: 'आ',
 23: 'इ',
 24: 'ई',
 25: 'उ',
 26: 'ऊ',
 27: 'ऋ',
 28: 'ए',
 29: 'ऐ',
 30: 'ऑ',
 31: 'ओ',
 32: 'औ',
 33: 'क',
 34: 'ख',
 35: 'ग',
 36: 'घ',
 37: 'च',
 38: 'छ',
 39: 'ज',
 40: 'झ',
 41: 'ञ',
 42: 'ट',
 43: 'ठ',
 44: 'ड',
 45: 'ढ',
 46: 'ण',
 47: 'त',
 48: 'थ',
 49: 'द',
 50: 'ध',
 51: 'न',
 52: 'प',
 53: 'फ',
 54: 'ब',
 55: 'भ',
 56: 'म',
 57: 'य',
 58: 'र',
 59: 'ल',
 60: 'व',
 61: 'श',
 62: 'ष',
 63: 'स',
 64: 'ह',
 65: '़',
 66: 'ा',
 67: 'ि',
 68: 'ी',
 69: 'ु',
 70: 'ू',
 71: 'ृ',
 72: 'ॅ',
 73: 'े',
 74: 'ै',
 75: 'ॉ',
 76: 'ो',
 77: 'ौ',
 78: '्',
 79: '।',
 80: '०',
 81: '१',
 82: '४',
 83: '५',
 84: '६',
 85: '७',
 86: '८',
 87: '९',
 88: '\u200d',
 89: '€'}

In [30]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [31]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

1/1 [==============================] - 0s 18ms/step
-
Input sentence: Wow!
Decoded sentence: मैं तुम्हें किताब है।

1/1 [==============================] - 0s 29ms/step
-
Input sentence: Duck!
Decoded sentence: मैं तुम्हें किताब है।

1/1 [==============================] - 0s 27ms/step
-
Input sentence: Duck!
Decoded sentence: मैं तुम्हें किताब है।

1/1 [==============================] - 0s 22ms/step
-
Input sentence: Help!
Decoded sentence: मैं तुम्हें किताब है।

1/1 [==============================] - 0s 17ms/step
-
Input sentence: Jump.
Decoded sentence: मैं तुम्हें किताब है।

1/1 [==============================] - 0s 17ms/step
-
Input sentence: Jump.
Decoded sentence: मैं तुम्हें किताब है।

1/1 [==============================] - 0s 20ms/step
-
Input sentence: Jump.
Decoded sentence: मैं तुम्हें किताब है।

1/1 [==============================] - 0s 22ms/step
-
Input sentence: Hello!
Decoded sentence: मैं तुम्हें किताब है।

1/1 [==============================] - 0s 21ms/step
-
Input sent